In [32]:
#from fla.layers.kda import KimiDeltaAttention as Attn
#from fla.layers.comba import Comba as Attn
from fla.modules.convolution import ShortConvolution
from fla.layers.rwkv7 import RWKV7Attention as Attn
import torch

In [33]:
batch, seq, dim = 1, 5, 4
num_heads = 2

assert dim % num_heads == 0
head_dim = dim // num_heads

In [34]:
# conv = ShortConvolution(dim, kernel_size=4)
attn = Attn(hidden_size=dim, num_heads=num_heads, head_dim=head_dim, fuse_norm=True)
attn2 = Attn(hidden_size=dim, num_heads=num_heads, head_dim=head_dim)

/media/john/Tertiary/Projects/ML/BayesianFlowNet/.venv/lib/python3.12/site-packages/fla/layers/rwkv7.py:143: UserWarning: According to Bo, you are using a potentially buggy FLA implementation of RWKV. If you plan to report any numbers based on this implementation, we strongly recommend cross-checking with the official repo: https://github.com/BlinkDL/RWKV-LM. Bo may disagree with results reported from this version.
  warnings.warn(


In [35]:
x = torch.randn(batch, seq, dim)

In [36]:
cu_seqlens = torch.tensor([0, 2, 5], dtype=torch.long).cuda()

In [37]:
# conv = conv.cuda()
x = x.cuda()
attn = attn.cuda()
attn2 = attn2.cuda()
v_first = torch.zeros_like(x)

In [47]:
attn.v_lora.lora[0].weight.dtype

torch.float32

In [38]:
output = attn(x, cu_seqlens=cu_seqlens, v_first=v_first)

/media/john/Tertiary/Projects/ML/BayesianFlowNet/.venv/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:1044: RuntimeWarning: ChunkDeltaRuleFunction does not support float32 on some platforms. Please use bfloat16/float16.
            If you want to use float32, please solve the issue by yourself.
  return fn(*args, **kwargs)


In [39]:
output

(tensor([[[ 0.1945,  0.0054,  0.0152, -0.0632],
          [-0.2154, -0.0184, -0.0145,  0.0664],
          [-0.0146,  0.1472, -0.0295,  0.0487],
          [ 0.2111,  0.0156,  0.0147, -0.0658],
          [-0.2097, -0.0210, -0.0135,  0.0637]]], device='cuda:0',
        grad_fn=<UnsafeViewBackward0>),
 None,
 None,
 tensor([[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]], device='cuda:0'))

In [40]:
attn(x[:, :2, :], v_first=v_first[:, :2, :])

(tensor([[[ 0.1945,  0.0054,  0.0152, -0.0632],
          [-0.2154, -0.0184, -0.0145,  0.0664]]], device='cuda:0',
        grad_fn=<UnsafeViewBackward0>),
 None,
 None,
 tensor([[[0., 0., 0., 0.],
          [0., 0., 0., 0.]]], device='cuda:0'))

In [41]:
attn(x[:, 2:, :], v_first=v_first[:, 2:, :])

(tensor([[[-0.0146,  0.1472, -0.0295,  0.0487],
          [ 0.2111,  0.0156,  0.0147, -0.0658],
          [-0.2097, -0.0210, -0.0135,  0.0637]]], device='cuda:0',
        grad_fn=<UnsafeViewBackward0>),
 None,
 None,
 tensor([[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]], device='cuda:0'))

In [42]:
output2 = attn2(output[0], cu_seqlens=cu_seqlens, v_first=output[-1])

In [43]:
output2

(tensor([[[ 8.9519e-04, -5.9764e-04, -1.5872e-03,  3.2160e-04],
          [-1.7288e-03,  1.1564e-03,  3.0754e-03, -6.5071e-04],
          [-2.5533e-04,  1.6467e-04,  4.2693e-04, -1.5897e-05],
          [ 1.3479e-03, -8.9255e-04, -2.3572e-03,  3.8792e-04],
          [-1.8802e-03,  1.2522e-03,  3.3200e-03, -6.3534e-04]]],
        device='cuda:0', grad_fn=<UnsafeViewBackward0>),
 None,
 None,
 tensor([[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]], device='cuda:0'))

In [44]:
o = output2[0]

In [45]:
o.mean().backward()